# Kirjastot

In [10]:


sijainti = input("Anna mallille nimi: ")
max_length = int(input("Anna opetettavien sekvenssien maksimipituus: "))
sijainti = "/content/gdrive/My Drive/models/" + sijainti + "_len_seq_" + str(max_length)
print("Mallin tallennuskansio drivessä: " + sijainti )

KeyboardInterrupt: ignored

In [2]:
!pip install --upgrade tensorflow


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 588.3 MB 6.0 kB/s 
     |████████████████████████████████| 1.7 MB 45.3 MB/s 
     |████████████████████████████████| 439 kB 47.5 MB/s 
     |████████████████████████████████| 6.0 MB 54.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.12
    Uninstalling flatbuffers-1.12:
      Successfully 

In [ ]:
#!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2
#!pip uninstall -y -q tensorflow keras tensorflow-estimator tensorflow-text
#!pip install -q tensorflow_datasets

In [3]:

!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.9 MB/s 


In [4]:
import logging
import time

import numpy as np
import matplotlib.pyplot as plt

import tensorflow_datasets as tfds
import tensorflow as tf

import tensorflow_text

# Datan nouto ja käsittely

In [5]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [6]:
def parse(filepath):
  parsed = []
  with open(filepath,'r') as data_file:
    lines = data_file.read().split("\n")[:-1]
    for line in lines:
      line = line.split(",")
      parsed.append(line)
  data_file.close()
  return parsed


aas = parse('/content/gdrive/My Drive/outputs.txt')
codons = parse('/content/gdrive/My Drive/inputs.txt')


In [7]:
codons_query = parse('/content/gdrive/My Drive/inputs.txt')

In [8]:
def shorten(seq, max_length):
  seq2 = []
  for i in seq:
    if len(i) > max_length:
      first = i[: (len(i) // 2)]
      second = i[(len(i) // 2) :]
      seq2.append(first)
      seq2.append(second)
    else:
      seq2.append(i)
  return seq2

In [9]:
while any(len(it) > max_length for it in aas):
    aas = shorten(aas, max_length)
while any(len(it) > max_length for it in codons):
    codons = shorten(codons, max_length)


NameError: ignored

In [ ]:
for i in codons:
  i.insert(0, "[START]")
  i.append("[END]")

for i in aas:
  i.insert(0, "[START]")
  i.append("[END]")

In [ ]:
pairs = list(zip(aas, codons))

In [ ]:
pairs[0]

In [ ]:
aas = [" ".join(i) for i in aas]
codons = [" ".join(i) for i in codons]
pairs = list(zip(aas, codons))

In [ ]:
def makeds(seqs):
  num_val_samples = int(0.07 * len(seqs))
  num_train_samples = len(seqs) - 2 * num_val_samples
  train_pairs = seqs[:num_train_samples]
  val_pairs = seqs[num_train_samples :]
  return (train_pairs, val_pairs)

In [ ]:
train_aas, val_aas = makeds(aas)
train_codons, val_codons = makeds(codons)

In [ ]:
ds = tf.data.Dataset.from_tensor_slices((aas, codons))

tds = tf.data.Dataset.from_tensor_slices((train_aas, train_codons))
vds = tf.data.Dataset.from_tensor_slices((val_aas, val_codons))

In [ ]:
aa2id = tf.keras.layers.TextVectorization(standardize = None, split = "whitespace")
aa2id.adapt(tf.constant(aas))


codon2id = tf.keras.layers.TextVectorization(standardize = None, split = "whitespace")
codon2id.adapt(tf.constant(codons))


In [ ]:
codon_voc = codon2id.get_vocabulary()

In [ ]:
id2codon = {str(codon_voc.index(i)) : i for i in codon_voc}

In [ ]:
def decode_codons(result):
  return [id2codon[str(key)] for key in np.array(result)]

In [ ]:
import pickle

with open('codon2id.pickle', 'rb') as handle:
    codon2id = pickle.load(handle)

# Inference

In [ ]:
sijainti = "/content/gdrive/My Drive/models/visvatron_150bpattention_kunnon"

In [ ]:
print("Testataan lataus")
reloaded = tf.saved_model.load(f'{sijainti}')
print("Kohde ladattu:" , reloaded)

In [ ]:
def print_translation(sentence, tokens, ground_truth):
  print(f'{"Input:":15s}: {sentence} length: {len(sentence.split(" "))}')
  print(f'{"Prediction":15s}: {tokens}, length: {len(tokens)}')
  print(f'{"Ground truth":15s}: {ground_truth}')

In [ ]:
print("Testataan input")

sentence = 'M K R I S T T I T T T I T I T T G N G A G *'
sentence = "[START] " + sentence + " [END]"
ground_truth = 'ATG AAA CGC ATT AGC ACC ACC ATT ACC ACC ACC ATC ACC ATT ACC ACA GGT AAC GGT GCG GGC TGA'.upper()

translated_text, translated_tokens, attention_weights = reloaded(sentence)
print_translation(sentence, translated_text, ground_truth)

In [ ]:
!pip install biopython

In [ ]:
from random import randrange
import statistics
from Bio.Seq import Seq

def print_translation(sentence, tokens, ground_truth, alignment, translated_pred, aa_alignment):
  print(f"Optimoidaan: {sentence}")
  print("---------------------------------------------------------------------------------")
  print(f'{"Ennuste":25s}: {" ".join(tokens.split(" ")[1: -1])}, length: {len(tokens.split(" "))}')
  print(f'{"Alignment":25s}: {" ".join(alignment[1: -1])}')
  print(f'{"Alkuperäinen":25s}: {" ".join(ground_truth.split(" ")[1: -1])}')
  print(f'{"Codon alignment tarkkuus":25s}: {round(accuracy, 2)}%')
  print("---------------------------------------------------------------------------------")
  print(f'{"Ennuste":25s}: {translated_pred}')
  print(f'{"Alignment":25s}: {" ".join(aa_alignment)}')
  print(f'{"Aminohapot:":25s}: {" ".join(sentence.split(" ")[1: -1])} length: {len(sentence.split(" "))}')
  print(f'{"AA alignment tarkkuus":25s}: {round(aa_accuracy, 2)}%')

def sample_pair(pairs, length):
  samples = [i for i in pairs if len(i[0]) < length]
  test_pair = samples[randrange(len(samples))]
  aa = test_pair[0]
  codon = test_pair[1]
  return aa, codon

def align(prediction, ground_truth):
  alignment = ["|||" if x == y else "XXX" for x, y in zip(prediction.split(" "), ground_truth.split(" "))]
  accuracy = statistics.mean([1 if x == y else 0 for x, y in zip(prediction.split(" "), ground_truth.split(" "))])
  return alignment, accuracy * 100
  
def aa_align(prediction, ground_truth):
  alignment = ["|" if x == y else "X" for x, y in zip(prediction.split(" "), ground_truth.split(" "))]
  accuracy = statistics.mean([1 if x == y else 0 for x, y in zip(prediction.split(" "), ground_truth.split(" "))])
  return alignment, accuracy * 100

In [ ]:
def plot_attention_head(in_tokens, translated_tokens, attention):
  # The model didn't generate `<START>` in the output. Skip it.
  translated_tokens = translated_tokens[1:]

  ax = plt.gca()
  ax.matshow(attention)
  ax.set_xticks(range(len(in_tokens)))
  ax.set_yticks(range(len(translated_tokens)))

  labels = [label for label in decode_aas(in_tokens.numpy())]
  ax.set_xticklabels(
      labels, rotation=90)

  labels = [label for label in decode_codons(translated_tokens.numpy())]
  ax.set_yticklabels(labels)

def plot_attention_weights(sentence, translated_tokens, attention_heads):
  in_tokens = aa2id(sentence)
  print(in_tokens)
  fig = plt.figure(figsize=(32, 16))

  for h, head in enumerate(attention_heads):
    ax = fig.add_subplot(2, 4, h+1)

    plot_attention_head(in_tokens, translated_tokens, head)

    ax.set_xlabel(f'Head {h+1}')

  plt.tight_layout()
  plt.show()

In [ ]:
backtranslated_aas, attention_weights, tokens = reloaded(sentence)

In [ ]:
for i in attention_weights[0]:
  print(i)

In [ ]:
print("Testataan input")


sentence, ground_truth = sample_pair(pairs, 100)
backtranslated_aas, attention_weights, tokens = reloaded(sentence)

backtranslated_aas = " ".join(list(decode_codons(backtranslated_aas)))

translated_pred = " ".join(list(Seq("".join(backtranslated_aas.split(" ")[1: -1])).translate()))
alignment, accuracy = align(backtranslated_aas, ground_truth)

aa_alignment, aa_accuracy = aa_align(translated_pred, " ".join(sentence.split(" ")[1: -1]))
print_translation(sentence, backtranslated_aas, ground_truth, alignment, translated_pred, aa_alignment)
print("#######################################################################################")
print("")
print("")
print("")
print("Mihin neuroverkko on kiinnittänyt huomiota, MULTI HEAD ATTENTION:")

plot_attention_weights(sentence,
                       tokens,
                       attention_weights[0])

In [ ]:
head = 0
# Shape: `(batch=1, num_heads, seq_len_q, seq_len_k)`.
attention_heads = tf.squeeze(attention_weights, 0)
attention = attention_heads[head]
attention.shape

In [ ]:
def plot_attention_weights(sentence, translated_tokens, attention_heads):
  in_tokens = aa2id(sentence)

  fig = plt.figure(figsize=(16, 8))

  for h, head in enumerate(attention_heads):
    ax = fig.add_subplot(2, 4, h+1)

    plot_attention_head(in_tokens, translated_tokens, head)

    ax.set_xlabel(f'Head {h+1}')

  plt.tight_layout()
  plt.show()

In [ ]:
print("Testataan attention plot:")

plot_attention_weights(sentence,
                       translated_tokens,
                       attention_weights[0])

In [ ]:
np.array(decode_codons(reloaded("[START] M K R I S T T I T T T I T I T T G N G A G * [END]")))

In [ ]:
print(pairs[0][1])
print(" ".join(decode_codons(reloaded("[START] M K R I S T T I T T T I T I T T G N G A G * [END]"))))

In [ ]:
print("Valmis! Heippa!")

In [ ]:
!tensorboard dev upload --logdir ./logs \
  --name "seppo" \
  --description f"tallennuspaikka: {sijainti}" \
  --one_shot

In [ ]:
!kill -9 -1